Little Lemon is a small family-owned Mediterranean restaurant.
They want to design a Python application that must link and store booking, menu, and orders information in MySQL database tables.
The business owner will use the stored data to drive revenue-making decisions.
Therefore, creating and manage a database is one of the Little Lemons goals.

In [1]:
from mysql.connector.pooling import MySQLConnectionPool
import mysql.connector as connector
from mysql.connector import Error

Establishing a connection to a database with the password "1234" and creating the Little Lemon database


In [2]:
try:
    connection = connector.connect(user="root", password="1234")
    cursor = connection.cursor()
    cursor.execute("CREATE DATABASE little_lemon")
    print("The database 'little_lemon' is created.\n")
    cursor.execute("USE little_lemon_db")
    connection.database
    print("The database is set for use.")
except Error as error:
    print(error.msg)

Can't create database 'little_lemon'; database exists


Creating a connection pool with 2 connections and a Function to execute SQL queries

In [3]:
dbconfig = {
    "database": "little_lemon",
    "user": "root",
    "password": "1234"
}

try:
    # Connection pool with 2 connections
    pool = MySQLConnectionPool(pool_name="pool_a", pool_size=2, **dbconfig)
    print("The connection pool is created with a name: ", pool.pool_name)
    print("The pool size is: ", pool.pool_size)

    # Function to execute SQL queries
    def execute_query(query, values=None):
        connection = pool.get_connection()  # Get a connection from the pool
        cursor = connection.cursor()
        try:
            cursor.execute(query, values)
            connection.commit()
            print("Query executed successfully.")
        except Error as e:
            print("Error executing query:", e)
        finally:
            cursor.close()
            connection.close()
except Error as er:
    print("Error:", er)

The connection pool is created with a name:  pool_a
The pool size is:  2


Creating the tables and populate with required data

In [4]:
# Create the booking table
create_booking_table_query = """
CREATE TABLE booking (
    booking_id INT AUTO_INCREMENT PRIMARY KEY,
    booking_slot TIME NOT NULL,
    guest_name VARCHAR(100) NOT NULL,
    assigned_to VARCHAR(100) NOT NULL
);
"""

# Create the menu table
create_menu_table_query = """
CREATE TABLE menu (
    item_id INT AUTO_INCREMENT PRIMARY KEY,
    item_name VARCHAR(100) NOT NULL,
    item_description TEXT,
    item_price DECIMAL(10, 2) NOT NULL
);
"""

# Create the orders table
create_orders_table_query = """
CREATE TABLE orders (
    order_id INT AUTO_INCREMENT PRIMARY KEY,
    booking_id INT NOT NULL,
    item_id INT NOT NULL,
    quantity INT NOT NULL,
    FOREIGN KEY (booking_id) REFERENCES booking(booking_id),
    FOREIGN KEY (item_id) REFERENCES menu(item_id)
);
"""

# Populate the booking table with sample data
populate_booking_table_query = """
INSERT INTO booking (booking_slot, guest_name, assigned_to) 
VALUES 
('15:00:00', 'Vanessa McCarthy', 'Giorgos Dioudis [Head Chef]'),
('17:30:00', 'Marcos Romero', 'Fatma Kaya [Assistant Chef]'),
('18:00:00', 'Anees Java', 'Jhon Millar [Receptionist]');
"""

# Populate the menu table with sample data
populate_menu_table_query = """
INSERT INTO menu (item_name, item_description, item_price) 
VALUES 
('Mediterranean Salad', 'Fresh vegetables with olive oil dressing', 12.99),
('Lemon Chicken', 'Grilled chicken with lemon and herbs', 15.99),
('Falafel Wrap', 'Chickpea patties with tahini sauce in a wrap', 9.99),
('Baklava', 'Traditional Mediterranean dessert with nuts and honey', 7.99);
"""

# Populate the orders table with sample data
populate_orders_table_query = """
INSERT INTO orders (booking_id, item_id, quantity) 
VALUES 
(1, 1, 2),
(2, 2, 1),
(3, 3, 3),
(3, 4, 2);
"""

# Execute queries
execute_query(create_booking_table_query)
print("Booking table is created.")
execute_query(create_menu_table_query)
print("Menu table is created.")
execute_query(create_orders_table_query)
print("Orders table is created.")
print("Inserting data in the Booking table.")
execute_query(populate_booking_table_query)
print("Inserting data in the Menu table.")
execute_query(populate_menu_table_query)
print("Inserting data in the Orders table.")
execute_query(populate_orders_table_query)

Error executing query: 1050 (42S01): Table 'booking' already exists
Booking table is created.
Error executing query: 1050 (42S01): Table 'menu' already exists
Menu table is created.
Error executing query: 1050 (42S01): Table 'orders' already exists
Orders table is created.
Inserting data in the Booking table.
Query executed successfully.
Inserting data in the Menu table.
Query executed successfully.
Inserting data in the Orders table.
Query executed successfully.


Implementing the functionalities of viewing bookings, adding new bookings, canceling bookings, viewing orders, adding new orderes, generating sales reports, and calculating revenue, with corresponding functions that execute SQL queries to retrieve, update, and analyze data from the database. 

In [9]:
# View Booking Function
def view_bookings():
    query = "SELECT * FROM booking"
    connection = pool.get_connection()
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Bookings:")
        for booking_id, booking_slot, guest_name, assigned_to in cursor.fetchall():
            print(f"Booking ID: {booking_id}, Slot: {booking_slot}, Guest Name: {guest_name}, Assigned To: {assigned_to}")
    except Error as e:
        print("Error executing query:", e)
    finally:
        cursor.close()
        connection.close()

# Function to add a new booking
def add_booking():
    booking_slot = input("Enter booking slot (HH:MM:SS): ")
    guest_name = input("Enter guest name: ")
    assigned_to = input("Enter assigned staff: ")
    
    query = "INSERT INTO booking (booking_slot, guest_name, assigned_to) VALUES (%s, %s, %s)"
    values = (booking_slot, guest_name, assigned_to)
    
    execute_query(query, values)
    print("New booking added successfully.")
    
# Cancel Bookings Function
def cancel_booking(booking_id):
    query = f"DELETE FROM booking WHERE booking_id = {booking_id}"
    execute_query(query)
    print(f"Booking with ID {booking_id} canceled successfully.")

# Function to view orders
def view_orders():
    query = """
    SELECT o.order_id, b.booking_slot, b.guest_name, m.item_name, o.quantity
    FROM orders o
    INNER JOIN booking b ON o.booking_id = b.booking_id
    INNER JOIN menu m ON o.item_id = m.item_id
    """
    connection = pool.get_connection()
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Orders:")
        for order_id, booking_slot, guest_name, item_name, quantity in cursor.fetchall():
            print(f"Order ID: {order_id}, Booking Slot: {booking_slot}, Guest Name: {guest_name}, Item: {item_name}, Quantity: {quantity}")
    except Error as e:
        print("Error executing query:", e)
    finally:
        cursor.close()
        connection.close()

# Function to add an order
def add_order():
    booking_id = input("Enter Booking ID: ")
    item_id = input("Enter Item ID: ")
    quantity = input("Enter Quantity: ")
    
    query = "INSERT INTO orders (booking_id, item_id, quantity) VALUES (%s, %s, %s)"
    values = (booking_id, item_id, quantity)
    
    execute_query(query, values)
    print("New order added successfully.")

# Generate Sales Reports Function
def generate_sales_report():
    query = """
    SELECT m.item_name, 
           SUM(o.quantity) AS total_quantity, 
           SUM(m.item_price * o.quantity) AS total_revenue
    FROM menu m
    INNER JOIN orders o ON m.item_id = o.item_id
    INNER JOIN booking b ON o.booking_id = b.booking_id
    GROUP BY m.item_name
    """
    connection = pool.get_connection()
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        if not results:
            print("No orders found.")
        else:
            print("Sales Report:")
            print("Item Name | Total Quantity | Total Revenue")
            for item_name, total_quantity, total_revenue in results:
                print(f"{item_name} | {total_quantity} | ${total_revenue}")
    except Error as e:
        print("Error executing query:", e)
    finally:
        cursor.close()
        connection.close()


# Calculate Revenue Function
def calculate_revenue():
    query = """
    SELECT SUM(m.item_price * o.quantity) AS total_revenue
    FROM menu m
    INNER JOIN orders o ON m.item_id = o.item_id
    INNER JOIN booking b ON o.booking_id = b.booking_id
    """
    connection = pool.get_connection()
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        total_revenue = cursor.fetchone()[0]
        print(f"Total revenue generated: ${total_revenue}")
    except Error as e:
        print("Error executing query:", e)
    finally:
        cursor.close()
        connection.close()


Creating a simple command-line interface (CLI) for users to interact with the functionalities

In [ ]:
# Import necessary modules
import sys

# Define the CLI interface function
def main_menu():
    print("Welcome to Little Lemon Menu Sales Report System")
    print("1. View Bookings")
    print("2. Add Booking")
    print("3. Cancel Booking")
    print("4. Generate Sales Report")
    print("5. Calculate Revenue")
    print("6. View Orders")
    print("7. Add Order")
    print("8. Exit")

    choice = input("Please enter your choice: ")

    if choice == "1":
        view_bookings()
    elif choice == "2":
        add_booking()
    elif choice == "3":
        booking_id = input("Enter the Booking ID to cancel: ")
        cancel_booking(booking_id)
    elif choice == "4":
        generate_sales_report()
    elif choice == "5":
        calculate_revenue()
    elif choice == "6":
        view_orders()
    elif choice == "7":
        add_order()
    elif choice == "8":
        print("Exiting the program.")
        sys.exit()
    else:
        print("Invalid choice. Please try again.")
    
    # After completing the chosen operation, return to the main menu
    main_menu()

# Call the main menu function to start the program
main_menu()

Welcome to Little Lemon Menu Sales Report System
1. View Bookings
2. Add Booking
3. Cancel Booking
4. Generate Sales Report
5. Calculate Revenue
6. View Orders
7. Add Order
8. Exit


Please enter your choice:  7
Enter Booking ID:  2
Enter Item ID:  1
Enter Quantity:  6


Query executed successfully.
New order added successfully.
Welcome to Little Lemon Menu Sales Report System
1. View Bookings
2. Add Booking
3. Cancel Booking
4. Generate Sales Report
5. Calculate Revenue
6. View Orders
7. Add Order
8. Exit


Please enter your choice:  6


Orders:
Order ID: 1, Booking Slot: 15:00:00, Guest Name: Vanessa McCarthy, Item: Mediterranean Salad, Quantity: 2
Order ID: 5, Booking Slot: 15:00:00, Guest Name: Vanessa McCarthy, Item: Mediterranean Salad, Quantity: 2
Order ID: 9, Booking Slot: 15:00:00, Guest Name: Vanessa McCarthy, Item: Mediterranean Salad, Quantity: 2
Order ID: 13, Booking Slot: 15:00:00, Guest Name: Vanessa McCarthy, Item: Mediterranean Salad, Quantity: 2
Order ID: 17, Booking Slot: 15:00:00, Guest Name: Vanessa McCarthy, Item: Mediterranean Salad, Quantity: 2
Order ID: 2, Booking Slot: 17:30:00, Guest Name: Marcos Romero, Item: Lemon Chicken, Quantity: 1
Order ID: 6, Booking Slot: 17:30:00, Guest Name: Marcos Romero, Item: Lemon Chicken, Quantity: 1
Order ID: 10, Booking Slot: 17:30:00, Guest Name: Marcos Romero, Item: Lemon Chicken, Quantity: 1
Order ID: 14, Booking Slot: 17:30:00, Guest Name: Marcos Romero, Item: Lemon Chicken, Quantity: 1
Order ID: 18, Booking Slot: 17:30:00, Guest Name: Marcos Romero, Item:

Please enter your choice:  5


Total revenue generated: $517.54
Welcome to Little Lemon Menu Sales Report System
1. View Bookings
2. Add Booking
3. Cancel Booking
4. Generate Sales Report
5. Calculate Revenue
6. View Orders
7. Add Order
8. Exit
